# Notes

**Question**: *What's TF Model Garden.*

**Aha**: A lib that allows you acess pre-build models, to adjust to your necessities.

**URL**: https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert

# Code

In [6]:
%%capture

!pip install -q opencv-python # why i need this?
!pip install tensorflow-text
!pip install -q tf-models-official

## Setup

### Lib

In [7]:
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_models as tfm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

### Resources

In [8]:
## this is how
## we acess
## Bert model

gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

## Load and preprocess the dataset

### Get the dataset from TensorFlow Datasets

In [9]:
batch_size=32
glue, info = tfds.load('glue/mrpc',
                       with_info=True,
                       batch_size=batch_size)

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/2.0.0. Subsequent calls will reuse this data.
AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'


In [10]:
print(info.features)
print('\n')
print("Classes:")
print(info.features['label'].names)

FeaturesDict({
    'idx': int32,
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'sentence1': Text(shape=(), dtype=string),
    'sentence2': Text(shape=(), dtype=string),
})


Classes:
['not_equivalent', 'equivalent']


In [11]:
example_batch = next(iter(glue['train']))

for key, value in example_batch.items():
  print(f"{key:9s}: {value[0].numpy()}")

idx      : 1680
label    : 0
sentence1: b'The identical rovers will act as robotic geologists , searching for evidence of past water .'
sentence2: b'The rovers act as robotic geologists , moving on six wheels .'


### Process Data

In [12]:
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
    lower_case=True)

In [13]:
tokens = tokenizer(tf.constant(["TensorFlow!"]))
tokens

<tf.RaggedTensor [[[23435, 12314], [999]]]>

In [14]:
special = tokenizer.get_special_tokens_dict()
special

{'vocab_size': 30522,
 'start_of_sequence_id': 101,
 'end_of_segment_id': 102,
 'padding_id': 0,
 'mask_id': 103}

In [15]:
max_seq_length = 64

packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_seq_length,
    special_tokens_dict = tokenizer.get_special_tokens_dict())

In [16]:
sentences1 = ["hello tensorflow"]
tok1 = tokenizer(sentences1)
tok1

<tf.RaggedTensor [[[7592], [23435, 12314]]]>

In [17]:
sentences2 = ["goodbye tensorflow"]
tok2 = tokenizer(sentences2)
tok2

<tf.RaggedTensor [[[9119], [23435, 12314]]]>

In [18]:
packed = packer([tok1, tok2])

for key, tensor in packed.items():
  print(f"{key:15s}: {tensor[:, :12]}")

input_word_ids : [[  101  7592 23435 12314   102  9119 23435 12314   102     0     0     0]]
input_mask     : [[1 1 1 1 1 1 1 1 1 0 0 0]]
input_type_ids : [[0 0 0 0 0 1 1 1 1 0 0 0]]


Put All Together

In [19]:
class BertInputProcessor(tf.keras.layers.Layer):
  def __init__(self, tokenizer, packer):
    super().__init__()
    self.tokenizer = tokenizer
    self.packer = packer

  def call(self, inputs):
    tok1 = self.tokenizer(inputs['sentence1'])
    tok2 = self.tokenizer(inputs['sentence2'])

    packed = self.packer([tok1, tok2])

    if 'label' in inputs:
      return packed, inputs['label']
    else:
      return packed

Apply to the dataset

In [20]:
bert_inputs_processor = BertInputProcessor(tokenizer, packer)

In [21]:
glue_train = glue['train'].map(bert_inputs_processor).prefetch(1)

In [22]:
example_inputs, example_labels = next(iter(glue_train))

In [23]:
for key, value in example_inputs.items():
  print(f'{key:15s} shape: {value.shape}')

print(f'{"labels":15s} shape: {example_labels.shape}')

input_word_ids  shape: (32, 64)
input_mask      shape: (32, 64)
input_type_ids  shape: (32, 64)
labels          shape: (32,)


In [24]:
plt.pcolormesh(example_inputs['input_word_ids'])

In [25]:
plt.pcolormesh(example_inputs['input_mask'])

In [26]:
plt.pcolormesh(example_inputs['input_type_ids'])

In [27]:
glue_validation = glue['validation'].map(bert_inputs_processor).prefetch(1)
glue_test = glue['test'].map(bert_inputs_processor).prefetch(1)

## Build, train and export the model

### Build the model

In [28]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())
config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [56]:
encoder_config = tfm.nlp.encoders.EncoderConfig({
    'type':'bert',
    'bert': config_dict
})

In [57]:
bert_encoder = tfm.nlp.encoders.build_encoder(encoder_config)
bert_encoder

In [55]:
bert_classifier = tfm.nlp.models.BertClassifier(network=bert_encoder, num_classes=2, )

In [32]:
bert_classifier(example_inputs, training=True).numpy()[:10]

array([[-0.5320878 ,  0.9695705 ],
       [-0.4182374 ,  0.98895204],
       [-0.36864024,  1.0823276 ],
       [ 0.5420164 ,  0.9451803 ],
       [-0.1331858 ,  0.9792325 ],
       [-0.23708574,  0.5729923 ],
       [ 0.28609204, -0.32386982],
       [-0.23204884,  0.7961407 ],
       [-0.5067148 ,  0.8174568 ],
       [-0.33655012, -0.08161402]], dtype=float32)

In [33]:
tf.keras.utils.plot_model(bert_encoder, show_shapes=True, dpi=1)

### Restore Weights

In [34]:
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
checkpoint.read(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

### Set up and Optimazer

In [35]:
# Set up epochs and steps
epochs = 2
batch_size = 32
eval_batch_size = 32

train_data_size = info.splits['train'].num_examples
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate=2e-5

In [39]:
# linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
#     initial_learning_rate=initial_learning_rate,
#     end_learning_rate=0,
#     decay_steps=num_train_steps)

In [64]:
warmup_schedule = tfm.optimization.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = 0,
    warmup_steps = warmup_steps
)

In [72]:
x = tf.linspace(0, num_train_steps, 1001)
y = [warmup_schedule(xi) for xi in x]
plt.plot(x,y)
plt.xlabel('Train step')
plt.ylabel('Learning rate')

Text(47.097222222222214, 0.5, 'Learning rate')

In [93]:
optimizer = tf.keras.optimizers.AdamW(
    learning_rate = warmup_schedule)

In [95]:
#metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
bert_classifier.compile(optimizer=optimizer)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adamw.AdamW object at 0x7aed512e2e50>